In [ ]:
import pandas as pd
import torch
import numpy as np

from model.module import THAN

In [ ]:
MODEL_SAVE_PATH = f'./saved_models/{args.prefix}-{args.data}.pth'
NUM_NEIGHBORS = 10

In [ ]:
def eval_one_epoch(model: THAN, batch_sampler, data):
    with torch.no_grad():
        model = model.eval()
        batch_sampler.reset()
        while True:
            batches, counts, classes = batch_sampler.get_batch_index()
            if counts is None or counts.sum()==0:
                break
            tiles = len(batches)
            l = int(counts.sum() * tiles)

            src_l_cut = np.empty(l, dtype=int)
            dst_l_cut = np.empty(l, dtype=int)
            ts_l_cut = np.empty(l, dtype=int)
            src_utype_l_cut = np.empty(l, dtype=int)
            dst_utype_l_cut = np.empty(l, dtype=int)
            etype_l = np.empty(l, dtype=int)
            lbls = np.empty(l)
            s_idx = 0
            for i, batch in enumerate(batches):
                e_idx = s_idx + int(counts[i] * tiles)
                src_l_cut[s_idx: e_idx] = np.tile(data.src_l[batch], tiles)
                dst_l_cut[s_idx: e_idx] = np.tile(data.dst_l[batch], tiles)
                ts_l_cut[s_idx: e_idx] = np.tile(data.ts_l[batch], tiles)
                src_utype_l_cut[s_idx: e_idx] = np.tile(data.u_type_l[batch],
                                                        tiles)
                dst_utype_l_cut[s_idx: e_idx] = np.tile(data.v_type_l[batch],
                                                        tiles)
                etype_slice = np.repeat(classes, len(batch))
                etype_l[s_idx: e_idx] = etype_slice
                lbls[s_idx: e_idx] = (etype_slice == classes[i]).astype(np.float64)
                s_idx = e_idx

            prob = model.link_contrast(src_l_cut, dst_l_cut, ts_l_cut,
                                       src_utype_l_cut, dst_utype_l_cut,
                                       etype_l, lbls, NUM_NEIGHBORS)

    return prob

In [ ]:
model = torch.load(MODEL_SAVE_PATH)